In [14]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 10**3)
pd.set_option('display.max_rows', 10**3)

In [15]:
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import gc
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.preprocessing import scale
import talib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import alphalens
import seaborn as sns
%matplotlib inline
sns.set_style('white')

In [16]:
from pathlib import Path
import pandas as pd
from utils import rank_stocks_and_quantile

top = 250  # parameters -> papermill
DATA_STORE = Path(f'data/{top}_dataset.h5')
dataset_key = 'data/YEAR_20220906_20230811'
label_col = 'TARGET_ret_fwd_01d' 
label = label_col + '_rank_quantiled'

with pd.HDFStore(DATA_STORE) as store:
    dataset = store[dataset_key]
    # print(dataset.index.levels[datetime_level].tz)

# Check and localize the datetime level of the MultiIndex to UTC if needed
datetime_level = 0  # Assuming the datetime is the first level
if dataset.index.levels[datetime_level].tz is None:
    localized_level = dataset.index.levels[datetime_level].tz_localize('UTC')
    dataset.index = dataset.index.set_levels(localized_level, level=datetime_level)

# Applying the rank_stocks_and_quantile function to the dataset
dataset_ranked = rank_stocks_and_quantile(dataset, TARGET_col=label_col)
print(dataset_ranked.info())

del dataset

# Assuming your DataFrame is named df
cols = dataset_ranked.columns.tolist()
# Populate the features list with column names starting with 'feature_'
features = [col for col in cols if col.startswith('FEATURE_')]
# Find the first column starting with 'target_' and set it as the label
label_cols = [col for col in cols if col.startswith('TARGET_')]
print(len(features))  # This will show all the columns starting with 'feature_'
print(label)  # This will show the first column starting with 'target_'

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 59607 entries, (Timestamp('2022-09-06 00:00:00+0000', tz='UTC'), 'AA') to (Timestamp('2023-08-11 00:00:00+0000', tz='UTC'), 'ZTS')
Columns: 600 entries, FEATURE_open to TARGET_ret_fwd_01d_rank_quantiled
dtypes: float32(360), float64(32), int32(198), int64(1), int8(9)
memory usage: 142.6+ MB
None
586
TARGET_ret_fwd_01d_rank_quantiled


In [17]:
# features

In [18]:
label_cols

['TARGET_ret_fwd_frac_order',
 'TARGET_ret_fwd_01d',
 'TARGET_ret_fwd_02d',
 'TARGET_ret_fwd_03d',
 'TARGET_ret_fwd_04d',
 'TARGET_ret_fwd_05d',
 'TARGET_ret_fwd_10d',
 'TARGET_ret_fwd_21d',
 'TARGET_ret_fwd_42d',
 'TARGET_ret_fwd_63d',
 'TARGET_ret_fwd_126d',
 'TARGET_ret_fwd_252d',
 'TARGET_ret_fwd_01d_rank',
 'TARGET_ret_fwd_01d_rank_quantiled']

In [19]:
target = 'TARGET_ret_fwd_frac_order'
# # Assuming you have your original dataframe named df ready
# alphalens_analysis = generate_alphalens_tearsheet(dataset_ranked, \
#     label_col=target, price_col='FEATURE_close')


# # alphalens_analysis
# import alphalens as al
# al.tears.create_full_tear_sheet(alphalens_analysis)

In [20]:
dataset_ranked

FEATURE_open  FEATURE_high  FEATURE_low  \
date                      ticker                                            
2022-09-06 00:00:00+00:00 AA         49.534199     50.391998    47.329800   
                          AAL        13.190000     13.260000    12.740000   
                          AAPL      156.210999    156.830002   153.434998   
                          ABBV      135.917007    137.945999   134.867996   
                          ABT       102.018997    103.442001   101.754997   
...                                        ...           ...          ...   
2023-08-11 00:00:00+00:00 WYNN      103.989998    103.989998    99.610001   
                          X          22.330000     22.730000    22.261000   
                          XOM       110.370003    112.070000   110.050003   
                          YUM       134.250000    134.449997   133.380005   
                          ZTS       188.479996    191.600006   188.179993   

                                  FEATURE_close  FEATURE_volume  \
date                      ticker                                  
2022-09-06 00:00:00+00:00 AA          47.788700    6.818136e+06   
                          AAL         13.220000    2.796638e+07   
                          AAPL       154.274002    7.383713e+07   
                          ABBV       136.203995    4.859588e+06   
                          ABT        102.227997    4.692471e+06   
...                                         ...             ...   
2023-08-11 00:00:00+00:00 WYNN       100.550003    3.148065e+06   
                          X           22.719999    6.708229e+06   
                          XOM        111.830002    1.404704e+07   
                          YUM        134.199997    8.080090e+05   
                          ZTS        190.830002    1.742931e+06   

                                  FEATURE_market_cap  FEATURE_sector  \
date                      ticker                                       
2022-09-06 00:00:00+00:00 AA            5.703249e+09             4.0   
                          AAL           1.022514e+10             0.0   
                          AAPL          2.779610e+12             3.0   
                          ABBV          2.686048e+11             2.0   
                          ABT           1.827505e+11             2.0   
...                                              ...             ...   
2023-08-11 00:00:00+00:00 WYNN          1.144238e+10             0.0   
                          X             5.065737e+09             4.0   
                          XOM           4.476770e+11             5.0   
                          YUM           3.760435e+10             0.0   
                          ZTS           8.818486e+10             2.0   

                                  FEATURE_ret_frac_order  FEATURE_ret_01d  \
date                      ticker                                            
2022-09-06 00:00:00+00:00 AA                    6.346544        -0.027208   
                          AAL                   1.220027         0.017706   
                          AAPL                 11.106913        -0.008216   
                          ABBV                 10.293886         0.009614   
                          ABT                   5.967541         0.002049   
...                                                  ...              ...   
2023-08-11 00:00:00+00:00 WYNN                 15.163733        -0.035214   
                          X                     4.126132         0.009778   
                          XOM                  12.413455         0.015529   
                          YUM                   9.539517        -0.001265   
                          ZTS                  21.730034         0.009149   

                                  FEATURE_ret_02d  FEATURE_ret_03d  \
date                      ticker                                     
2022-09-06 00:00:00+00:00 AA            -0.003534        -0.031730   
           

In [21]:
dataset_ranked.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 59607 entries, (Timestamp('2022-09-06 00:00:00+0000', tz='UTC'), 'AA') to (Timestamp('2023-08-11 00:00:00+0000', tz='UTC'), 'ZTS')
Columns: 600 entries, FEATURE_open to TARGET_ret_fwd_01d_rank_quantiled
dtypes: float32(360), float64(32), int32(198), int64(1), int8(9)
memory usage: 142.6+ MB


In [26]:
## Idea: Convrting returns as a image shapshot: IPCV project
import numpy as np
import math

# Assuming 'dataset_ranked' is your dataframe and has already been loaded into the current workspace.
unique_dates = dataset_ranked.index.get_level_values(0).unique()

images = []

# Determine the dimensions of the 2D image based on the number of tickers.
num_tickers = len(dataset_ranked.index.get_level_values(1).unique())
side_length = math.floor(math.sqrt(num_tickers))  # Largest square less than num_tickers

# Adjust the number of tickers
num_tickers = side_length * side_length

for idx, date in enumerate(unique_dates):
    day_data = dataset_ranked.loc[date].iloc[:num_tickers]  # Keep only the number of rows we need
    
    if all(feature in day_data.columns for feature in \
        ['FEATURE_ret_frac_order', 'FEATURE_ret_01d', 'FEATURE_ret_02d']):
        
        channels = []
        
        for feature in ['FEATURE_ret_frac_order', 'FEATURE_ret_01d', 'FEATURE_ret_02d']:
            # Convert the data to a 2D image representation
            feature_data = day_data[feature].values
            # Reshape it to a square matrix
            feature_image = feature_data.reshape(side_length, side_length)
            channels.append(feature_image)
        
        # Stack the channels to form a multi-channel image
        image = np.stack(channels, axis=-1)
        images.append(image)
    else:
        print(f"Skipping date {date} due to inadequate data.")

print("Number of images:", len(images))
if images:
    print(images[0].shape)  # This should give (side_length, side_length, 3)
else:
    print("No images created.")


Number of images: 235
(15, 15, 3)


In [23]:
print(images[0].shape)

(15, 15, 3)


In [24]:
from PIL import Image

# Get the first image from the images list
first_image = images[0]

# Scale it to 0-255 range for visualization (useful if your data isn't already in this range)
scaled_image = (first_image - first_image.min()) / (first_image.max() - first_image.min()) * 255
scaled_image = scaled_image.astype(np.uint8)

# Create a PIL image
image_pil = Image.fromarray(scaled_image)

# Display the image
image_pil.show()


In [25]:
from IPython.display import display


display(image_pil)

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/sayem/snap/code/common/.cache/gio-modules/libgiolibproxy.so


eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
